# Adat ellenőrzés (validáció)

Már megszokhattuk, hogy a python esetében össze-vissza "ragasztgathatjuk" a cimkéinket amire csak akarjuk. Az `x` jelenthet számot, három sorral odébb meg egy numpy tömböt vagy éppen egy fájl objektumot. A függvények paraméterénél ugyan megadhatunk tippeket (type hint) de az interpreter ezt nem kényszeríti ki, ettől még átadhatunk mást is.

Ez néha áldás, néha viszont átok. Az egyik gyakori eset amikor ez e legkevésbé sem hasznos az a szabványos kommunikáció. Ha egy másik rendszerből adatokat veszünk át, vagy át szeretnénk oda vinni, általában erősen kötött formátummal dolgozunk. Ha egyetlen szám szövegként van megadva, már fejre is áll minden.

Ha máshonnan jön adat (külső renszer, felhasználó) semmi garancia nincs rá, hogy jó formátumban jön az adat, azaz:
- minden mező megjön
- jó típusban jön (szám helyett string stb.),
- az érték tartomány érvényes (negatív darabszám, jövőbeli születési dátum, stb.).

Ha nincs egyértelmű, típusokkal leírt "szerződés", ami előírja mit kell átadni, akkor:

* a hibák futásidőben, gyakran későn derülnek ki,
* nehéz a hibát megtalálni,
* különböző részek másképp (félre) értelmezik ugyanazt az adatot (gondolj csak rá, mennyi mindent jelenthet pythonban a szorzásjel vagy a hatványjel!)


Ha szigorú típusokra van szükségünk (és azt ellenőrizni is szeretnénk) a legjobb barátunk a Pydantic könyvtár lesz.

## Pydantic
A Pydantic a Python típusannotációira épülő adat-validációs és adatmodell könyvtár. (Jól hangzott, mi?)  Tehát, megadod a típusaidat, a Pydantic pedig ellenőrzi, átalakítja, és garantálja, hogy a rendszeredben mozgó adatok megfelelnek az általad megadott szabályoknak. Nem lesz se kisebb, se nagyobb, se hosszabb, se semmilyen módon más.

A pydantic nem beépített csomag tehát fel kell tenned:
```bash
pip install pydantic
```

Itt a colabban minden fontos csomag fenn van, így ez is.


## Modellek

In [ ]:
# a pydantic minden "ellenőrizhető" struktúrát a BaseModell-ből származtat
from pydantic import BaseModel
from datetime import date

class Person(BaseModel):
    name: str
    age: int
    birth_date: date | None = None

p = Person(name="Anna", age="30", birth_date="1995-05-10")
print(p)
print(p.age, type(p.age))
print(p.birth_date, type(p.birth_date))


Mit csinálunk itt?
* "30" magától átalakul `int` típussá
* "1995-05-10" `date` típussá
* Ha pedig zöldséget adunk meg (age="harminc") akkor `ValidationError`-t kapunk

Készíts egy programot, ami a felhasználótól bekéri ezt a három adatot vesszővel elválasztva és ellenőrzi ("validálja")! Fogd el a ValidationError hibát, és írd, ki hogy rossz adatot adott meg.

Szétvágni a szöveget így tudod: `szöveg.split(',')`.

In [ ]:
# felhasználó adat érvényesítés
...



## Kötelező és opcionális mezők, alapértékekkel

In [ ]:
from pydantic import BaseModel
from typing import Optional

class SensorReading(BaseModel):
    id: int
    value: float
    unit: str = "C"  # alapértelmezett érték
    location: Optional[str] = None  # opcionális

r1 = SensorReading(id=1, value=23.5)
r2 = SensorReading(id=2, value=18.2, unit="kPa", location="Lab 3")
r1,r2

(SensorReading(id=1, value=23.5, unit='C', location=None),
 SensorReading(id=2, value=18.2, unit='kPa', location='Lab 3'))

## Összetett struktúrák

Interfészeknél gyakori, hogy JSON-ban beágyazott objektumok vannak (azokban további beágyazott objektumok és így tovább).

In [ ]:
from pydantic import BaseModel
from typing import List

# legyen egy pont típusunk, ami egy lebegőpontos számpár.
class Pont(BaseModel):
    x: float
    y: float

class Mérés(BaseModel):
    id: int
    pontok: List[Pont]
    leírás: str | None = None # vagy van és szöveg vagy nincs

# bejövő adat (érvényesítés).
# próbálj meg benne hibát csinálni: pl y helyett z vagy rossz adat!
m = Mérés(
    id=42,
    pontok=[
      {"x": 0, "y": 1.5},
      {"x": 2.3, "y": -0.4},
    ],
)

m

De mi a helyzet ha a két adattípus nem együtt érkezik, hanem vagy az egyik, vagy a másik? Tehát mondjuk egy felhasználói azonosító lehet, hogy `int` de lehet, hogy `str`!

Nos, simán csak megadjuk mint alternatív python típus:

In [ ]:
class User(BaseModel):
    id: str | int

User(id=12), User(id="jozsi")

*Extra infó*: a programozók (és a matematikusok) úgy hívják ezt, hogy 'sum type' mivel a lehetséges a két típus elemeinek összege (összes str + összes int).

Ezzel szemben, ha mindkettőt tárolod, például:
```python
class User(BaseModel):
  id:int
  név:str
```
Akkor az egy "product type" lenne, mivel a lehetséges értékei a két típus szorzata. (Összes int kombinálható az összes str-el).

## Érvényességi szabályok

Gyakran nem elég nekünk az, hogy a típus rendben van, hanem pontosan meg szeretnénk adni az érvényességi tartományt (min/max, hossz, stb).

Ez esetben használhatjuk a Field objektumot, ahol mindez megadható!

In [ ]:
from pydantic import BaseModel, Field

class Product(BaseModel):
  name: str = Field(min_length=1, max_length=100)
  price: float = Field(gt=0)  # > 0, gt = greater
  quantity: int = Field(ge=0) # >= 0, ge = greater equal

Product(name="Cement", price=12.5, quantity=100)

In [ ]:
# ha a termék neve nem lehet bármi, használhatunk Literal típust
# (konkrét értékkészlet)
from typing import Literal

Terméktípus = Literal["Cement", "Steel"]

class Product(BaseModel):
  name: Terméktípus
  price: float = Field(gt=0)  # > 0, gt = greater
  quantity: int = Field(ge=0) # >= 0, ge = greater equal

Product(name="Steel", price=12.5, quantity=100)

In [ ]:
# de írhatunk saját érvényességi szabályt is
class Ember(BaseModel):
  name: str
  age: float = Field(gt=0, le=200)
  def name_validator()

## Adat ki/be menet, kommunikáció JSON-el

Tegyük fel, hogy a korábbi szigorúan definiált szabvánnyal (interfésszel) dolgozunk és az elábbi adatot kapjuk (felhasználótól, API hívástól, más rendszerből):

In [ ]:
class Product(BaseModel):
  name: str = Field(min_length=1, max_length=100)
  price: float = Field(gt=0, le=10000)
  quantity: int = Field(ge=0)

bejövő = {
    "name": "Cement",
    "price": "12.5",
    "quantity": "100"
}

In [ ]:
# mivel ez eleve python struktúra, az eset szuper egyszerű:
p = Product(**bejövő)

# és már használhatjuk is:
p.name, p.price, p.quantity

('Cement', 12.5, 100)

Sajnos nekünk általában nem python kódként jön az adat, hanem más formában, pl CSV vagy még gyakrabban JSON. De ez sem probléma!



In [ ]:
# ez egy szöveg, pl fájlból jön vagy hálózaton:
json_adat = '{"name": "Cement", "price": "12.5", "quantity": "100"}'

In [ ]:
#validáljuk az adatot:
Product.model_validate_json(json_adat)

De az is könnyen előfordulhat, hogy sok ilyen "Product" adatunk van, például egy listában! Mi pedig mindet validálni szeretnénk.  De legyen benne egy kis csavar: a hibás adatokat egyszerűen ignoráljuk.

Normál esetben ha bármelyik adat bárhol rossz, azonnal ValidationError-t kapunk, hiszen a Pydantic (mint neve is mutatja) szuper szigorú!

In [ ]:
be_JSON = """
    [
      {"name": "Cement", "price": "12.5", "quantity": "100"},
      {"name": "Steel", "price": "22.1", "quantity": "-80"},
      {"name": "Cement", "price": "11.5", "quantity": "72"}
    ]
"""
# a harmadik sorban rosszul van megadva a mennyiség!


Ilyenkor segít nekünk a TypeAdapter, ami egy szokásos python tárolót
(pl. listát) és egy Pydantic típust tud kombinálni. A TypeAdapter már nem lesz Pydantic típus (más típusokban már nem tudod használni), egyszerűen csak abban segít nekünk, hogy könnyen tudjunk "listaolvasót" vagy "listaírót" csinálni.

A kihagyást az OnErrorOmit típussal fogjuk megoldani (ez valóban típus) aminek annyi képessége van, hogy a beágyazott típus hibája esetén nem dob hibát, csak kihagyja.


In [ ]:
from pydantic import OnErrorOmit, TypeAdapter

# készítsünk egy python lista-olvasót ami Product-okat vár:
adapter = TypeAdapter(list[OnErrorOmit[Product]])

# és már olvashatjuk is be (validálva!)
products = adapter.validate_json(be_JSON)

products

[Product(name='Cement', price=12.5, quantity=100),
 Product(name='Cement', price=11.5, quantity=72)]